# Model AOPs

First attempt at modeling AOPs.  For not will now use hill curves because the data is very noisy and don't know how
to assess goodness-of-fit.

Instead a simple approach could be done as follows

1) get chemical response data for target
2) use curveP to correct noise and establish monotonicity
3) put all concentrations on same scale

Imports...

In [1]:
import pandas as pd
import config
import seaborn as sns
import numpy as np
from curvep import curveP, CONCLIST
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau


sns.set(style="ticks", context="talk")
plt.style.use("dark_background")



### Get data

First get all the AIDs associated with a set of targets.

This part can be modified once its decided how assays will be grouped.  E.g, here ESR1, ESR2 will represent two different AOPs, pathways, etc.

In [13]:
# get all the targets in the database

q = "select GeneSymbol, count(distinct PUBCHEM_AID) as num_assay from targets group by GeneSymbol  ORDER BY num_assay DESC "

target_list = pd.read_sql_query(q, con=config.Config.DB_URI)
target_list = target_list.query("num_assay > 1 and GeneSymbol")
target_list.head()

,GeneSymbol,num_assay
1,Kcnq2,26
2,Scarb1,23
3,LOC116160065,21
4,CASP3,20
5,GBA,20


In [14]:
target_list.shape

(442, 2)

In [15]:
TARGETS = [t for t in target_list.GeneSymbol if t]
TARGETS

['Kcnq2',
 'Scarb1',
 'LOC116160065',
 'CASP3',
 'GBA',
 'PKM',
 'OPRK1',
 'MITF',
 'S1PR4',
 'APLNR',
 'HRAS',
 'OPRM1',
 'PPARG',
 'ATAD5',
 'Kcnj2',
 'NFE2L2',
 'OPRD1',
 'ALPL',
 'PADI4',
 'TAAR1',
 'CHRM5',
 'Tb10.70.6470',
 'ALPG',
 'ALPI',
 'ESR1',
 'GPR35',
 'PPP5C',
 'S1PR2',
 'AGTR1',
 'Chrm1',
 'PKLR',
 'PTPN22',
 'ABHD5',
 'Alpi',
 'KCNH2',
 'NOX1',
 'TSHR',
 'Trpc4',
 'UBE2N',
 'VCP',
 'ADRB2',
 'CHRM3',
 'Chrm4',
 'GAA',
 'GLA',
 'GNA15',
 'MTOR',
 'PPP1CA',
 'S1PR1',
 'S1PR3',
 'THRB',
 'APOBEC3G',
 'ATM',
 'CASP1',
 'CFTR',
 'CHRM2',
 'CLK4',
 'DUSP3',
 'HTT',
 'Hsf1',
 'KEAP1',
 'NPSR1',
 'PF3D7_0932300',
 'Tb927.3.3270',
 'USP2',
 'VDR',
 'APAF1',
 'AR',
 'CBFB',
 'CNR1',
 'ESRRA',
 'FASN',
 'GMNN',
 'IDH1',
 'KCNQ1',
 'NPBWR1',
 'PF3D7_1311800',
 'PLIN5',
 'RAC1',
 'RUNX1',
 'Rorc',
 'SMAD3',
 'TP53',
 'CASP9',
 'CHRM4',
 'GALK1',
 'GRM4',
 'GSK3B',
 'Grm5',
 'HSP90AA1',
 'IFNB1',
 'KMT2A',
 'LA59_RS18165',
 'MCHR1',
 'MCL1',
 'PADI1',
 'PADI2',
 'PADI3',
 'PAX8',
 '

In [16]:
# get AIDs associated with target
# then get active compounds

TARGETS_STR_LIST = ["\'" + t + "\'" for t in TARGETS]
TARGETS_STR = ", ".join(map(str, TARGETS_STR_LIST))
targets_array = f'({TARGETS_STR})'

# get assays that belong to a particular
# target
query = 'SELECT tG.PUBCHEM_AID, tG.GeneSymbol ' \
        'FROM targets tG ' \
        'WHERE GeneSymbol in {}' \
        ''.format(targets_array)

genes_aids = pd.read_sql_query(query, con=config.Config.DB_URI)

# get active compounds in AIDS

aid_list = [str(aid)for aid in genes_aids.PUBCHEM_AID]

aid_string = ", ".join(map(str, aid_list))
aid_query = f'({aid_string})'

actives_query = 'SELECT c.PUBCHEM_CID as CID, c.PUBCHEM_AID as AID, c.PUBCHEM_SID as SID ' \
                'FROM concise c ' \
                'WHERE c.PUBCHEM_AID in {} AND c.PUBCHEM_ACTIVITY_OUTCOME == "Active" AND ' \
                'c.PUBCHEM_CID is not null AND c.PUBCHEM_SID is not null'.format(aid_query)

active_cmps = pd.read_sql_query(actives_query, con=config.Config.DB_URI)
active_cmps['SID'] = active_cmps['SID'].astype(int)




In [17]:
active_cmps

,CID,AID,SID
0,1663,411,11111287
1,1730,411,11110959
2,1742,411,11111292
3,2392,411,4254198
4,2703,411,11110929
...,...,...,...
541887,135442941,1671201,144212405
541888,135491728,1671201,251919717
541889,135565635,1671201,170466897
541890,135585373,1671201,144211426


In [29]:
counts = active_cmps.groupby('AID')['CID'].nunique()
counts = counts[counts > 500]

In [30]:
trimmed_actives = active_cmps[active_cmps.AID.isin(counts.index)]
trimmed_actives.shape

(455212, 3)

In [32]:
counts = active_cmps.groupby('CID')['AID'].nunique()
counts = counts[counts > 5]

In [33]:
trimmed_actives = active_cmps[active_cmps.CID.isin(counts.index)]
trimmed_actives.shape

(212247, 3)

In [34]:
trimmed_actives.CID.nunique()

20583

In [35]:
sid_list = [str(sid)for sid in trimmed_actives.SID.unique()]

sid_string = ", ".join(map(str, sid_list))
sid_query = f'({sid_string})'

aid_list = [str(aid)for aid in trimmed_actives.AID.unique()]

aid_string = ", ".join(map(str, aid_list))
aid_query = f'({aid_string})'

hill_params_query = 'SELECT  SID, AID, AC50, TOP, SLOPE, MSE ' \
                    'FROM hill_models ' \
                    'WHERE AID in {} AND SID in {} '.format(aid_query, sid_query)

hill_params = pd.read_sql_query(hill_params_query, con=config.Config.DB_URI)

# this merge is necessary because
# the former query gathers all data
# for all sid and aid that have any active cmps
# not just the pairs of active sid, aid
# could be solved by a SQLite join, but
# right now takes too long
hill_merged = pd.merge(hill_params, trimmed_actives[['SID', 'CID', 'AID']].drop_duplicates(['SID', 'CID', 'AID']), on=['SID', 'AID'])
print(hill_merged.head())


      SID     AID      AC50         TOP     SLOPE           MSE     CID
0  842160    1688  9.950002  -69.010063  7.972477  2.955673e-09  644416
1  842160  504467  9.110092 -100.000249  8.000000  1.546978e-08  644416
2  842186    1688  0.369508 -104.586537  0.851785  6.308208e+01  644442
3  842186    2546  7.473386  -86.463212  7.999999  3.973917e-07  644442
4  842186    2551  1.997860 -100.000131  7.978447  8.293890e-09  644442


In [36]:
hill_merged = hill_merged.dropna()
hill_merged_trim = hill_merged[hill_merged.MSE < 500]
hill_merged_trim.shape

(202104, 7)

Now, create an example model just using one compound....

This can be extended to create models for $n$ compounds when ready...dd

In [37]:
hill_merged_trim.query("CID == 73864")

,SID,AID,AC50,TOP,SLOPE,MSE,CID
122479,26757531,588513,0.011761,-3.887146e+01,0.300000,1.635351e+02,73864
122480,26757531,588514,0.502084,4.824680e+01,2.213343,1.022685e+02,73864
122481,26757531,651741,45.388852,4.851048e+01,8.000000,4.311081e-07,73864
177782,144208665,651631,57.295229,-1.084922e+02,8.000000,1.739599e+00,73864
177783,144208665,743035,24.625737,-8.463526e+01,2.412551,1.665934e+00,73864
177784,144208665,743065,17.567024,-9.151778e+01,8.000000,1.758873e+02,73864
177785,144208665,743075,1.865656,1.780010e+01,7.999998,2.675787e+01,73864
177787,144208665,1159521,39.553445,-8.686703e+01,1.781940,3.631663e+00,73864
177788,144208665,1224839,12.899834,3.953936e+01,1.410130,5.433597e+00,73864
177789,144208665,1224841,57.433383,-1.200000e+02,3.022719,1.098316e+01,73864


In [38]:
scaled_data_corrected = hill_merged_trim.merge(genes_aids.rename(columns={'PUBCHEM_AID': 'AID', 'GeneSymbol': 'Target'}))

training_data = scaled_data_corrected.copy().dropna()
training_data.shape

(341635, 8)

In [39]:
training_data = training_data.drop_duplicates()
training_data.head()

,SID,AID,AC50,TOP,SLOPE,MSE,CID,Target
0,842160,1688,9.950002,-69.010063,7.972477,2.955673e-09,644416,HTT
1,842186,1688,0.369508,-104.586537,0.851785,6.308208e+01,644442,HTT
2,842939,1688,1.164601,-86.507807,1.079740,4.174652e+00,645210,HTT
3,843298,1688,9.327466,-71.745416,7.997913,8.245720e-09,645578,HTT
4,844255,1688,5.236478,-81.819205,1.302995,1.890081e+00,646579,HTT


Format data

In [40]:
from curve_fitting import hill_curve
xs = np.asarray(CONCLIST)

dfs = []

for (aid, cid), params in training_data.groupby(['AID', 'CID']):

    params = params[params.MSE == params.MSE.min()]

    curve = hill_curve(xs, params.AC50.iloc[0], params.TOP.iloc[0], params.SLOPE.iloc[0])
    df = pd.DataFrame()
    df['log(Concentration)'] = np.log10(xs)
    df['Response'] = curve
    df['AID'] = aid
    df['CID'] = cid
    df['Target'] = params.Target.iloc[0]
    dfs.append(df)

In [41]:
dfs = pd.concat(dfs)
dfs.head()

,log(Concentration),Response,AID,CID,Target
0,-6.000000,-5.683894e-09,411,1730,LOC116160065
1,-5.571924,-2.196190e-08,411,1730,LOC116160065
2,-5.395833,-3.829530e-08,411,1730,LOC116160065
3,-5.219742,-6.677606e-08,411,1730,LOC116160065
4,-5.043650,-1.164384e-07,411,1730,LOC116160065


In [42]:
# create the F matrix

F_matrix = pd.DataFrame(index=genes_aids.PUBCHEM_AID.unique().astype(int), columns=genes_aids.GeneSymbol.unique())

for aop, aop_data in genes_aids.groupby('GeneSymbol'):
    for aid in aop_data.PUBCHEM_AID.unique():
        F_matrix.loc[aid, aop] = 1
F_matrix = F_matrix.fillna(0)
F_matrix


,ABCB1,ABCB6,ABCG2,ABHD5,ABL1,ACHE,ACP1,ADAM10,ADAM17,ADRB2,...,mazF,mex-5,recA,recB,recC,recD,rev,skn-1,trpn1,vif
1689,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489002,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
504566,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
504569,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1508636,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1538,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1682,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2583,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
652208,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [43]:
frames = {}
cids = []


for cid, d in dfs.groupby('CID'):
    training_wide = d.pivot(columns='log(Concentration)', index='AID', values='Response')
    # fill missing data AIDs
    training_wide = training_wide.reindex(F_matrix.index).fillna(0)
    frames[cid] = training_wide
    cids.append(cid)

In [44]:
cr_responses = np.stack([frames[cas].values for cas in cids], axis=2)

In [45]:
cr_responses.shape

(2050, 45, 20573)

In [46]:
# check scales, will need to normalize to 0-1 eventually

training_wide.max(1)

1689       0.0
489002     0.0
504566     0.0
504569     0.0
1508636    0.0
          ... 
1538       0.0
1682       0.0
2583       0.0
652208     0.0
652241     0.0
Length: 2050, dtype: float64

In [ ]:

EPOCHS = 750
STEPS = 100
INITIAL_LR = 1
GAMMA = 0.5

F = torch.tensor(F_matrix.values, dtype=torch.float)
R = torch.tensor(np.random.random(size=(F.shape[1], cr_responses.shape[1], cr_responses.shape[2])),
                 dtype=torch.float, requires_grad=True)

# R = torch.tensor(torch.clamp(R, min=0, max=1), requires_grad=True)


y = torch.tensor(cr_responses)

optimizer = optim.Adam([R], lr=INITIAL_LR)
#scheduler = StepLR(optimizer, step_size=STEPS, gamma=0.9)
scheduler = ReduceLROnPlateau(optimizer, factor=0.1, patience=50)

In [ ]:
losses = np.array([])

for epoch in range(EPOCHS):
    # model is y = FR
    optimizer.zero_grad()
    #R = torch.clamp(R, min=0, max=1)
    model = torch.tensordot(F, R, dims=([1], [0]))


    ss_term = torch.mean(torch.square(model - y))

    x = torch.sum(R)
    penalty_term = x ** 10 / (x ** 10 + 0.5 ** 10)

    loss = ss_term + penalty_term
    loss = ss_term

    loss.backward()
    optimizer.step()
    losses = np.append(losses, loss.detach().numpy())
#     if losses[-50:].var() < 0.1 and epoch>10:
#         #print("Reducing LR...")
#         for g in optimizer.param_groups:
#             g['lr'] = g['lr']*GAMMA
    scheduler.step(loss)
    if epoch % STEPS == 0:
        print(loss)
        print('Epoch-{0} lr: {1}'.format(epoch, optimizer.param_groups[0]['lr']))

In [ ]:
plt.plot(losses)
plt.show()

In [ ]:
from curve_fitting import auc_score

scores = R.detach().numpy()

cmp_score_dic = {}

for i, casnumber in enumerate(cids):
    cmp_r_scores = pd.DataFrame(scores[:, :, i], index=F_matrix.columns, columns=np.log10(xs))
    cmp_auc_scores = auc_score(cmp_r_scores)
    cmp_score_dic[casnumber] = (cmp_r_scores, cmp_auc_scores)

In [ ]:
auc_score(cmp_r_scores)

In [ ]:
d = [cmp_score_dic[cas][1] for cas in cids]

score_frame = pd.DataFrame(d, index=cids)

In [ ]:
score_frame

In [ ]:
score_frame_norm = score_frame.divide(score_frame.max()).fillna(0)

scores_norm = scores.copy()

for r in range(scores_norm.shape[0]):
    scaler = scores[r, :, :].max()
    if scaler != 0:
        scores_norm[r, :, :] = scores_norm[r, :, :]*1/scaler

cmp_score_norm_dic = {}

for i, casnumber in enumerate(cids):
    cmp_r_scores = pd.DataFrame(scores_norm[:, :, i], index=F_matrix.columns, columns=np.log10(xs))
    cmp_score_norm_dic[casnumber] = cmp_r_scores

In [ ]:
ranked = score_frame_norm.sort_values('ESR1', ascending=False)
ranked['ESR1_Rank'] = list(range(1, ranked.shape[0]+1))

In [ ]:
ax = sns.scatterplot(data=ranked, x='ESR1_Rank', y='ESR1', s=10)
ax.axhline(0, ls='--', lw=2, color='r')

In [ ]:
ranked2 = score_frame_norm.sort_values('TP53', ascending=False)
ranked2['TP53_Rank'] = list(range(1, ranked2.shape[0]+1))

ax = sns.scatterplot(data=ranked2, x='TP53_Rank', y='TP53', s=10)
ax.axhline(0, ls='--', lw=2, color='r')

In [ ]:
target_cid = ranked2.index[-1]
target_cid

In [ ]:
ranked2

In [ ]:
target = TARGETS[1]
data = dfs.query(f'CID == {target_cid} and Target == "{target}"').pivot(columns='log(Concentration)', index='AID', values='Response').T
sns.lineplot(data=data, palette='Accent')
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
plt.title(f"Avg. Dose Responses for\nTarget {target} and CID {target_cid}")

In [ ]:
r = cmp_score_norm_dic[target_cid].T
r.head()

In [ ]:


sns.lineplot(data=r, palette='Accent')
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
plt.title(f"AOP Curve for\nTarget {target} and CID {target_cid}")

In [ ]:

target_cid

In [ ]:
score_frame_norm.loc[target_cid]

In [ ]:
auc_score(r.T)

In [ ]:
score_frame.max()

In [ ]:
-0.029  * (1 / 14)

In [ ]:
np.sign(r.T.diff(axis=1))

In [ ]:
r.T.sum(axis=1)

In [ ]:
signss = cmp_r_scores.diff(axis=1)
np.sign(cmp_r_scores.sum(1))

In [ ]:
cmp_r_scores.diff(axis=1)

In [ ]:
-1 - 1

In [ ]:
sns.lineplot(data=cmp_r_scores.T)

In [ ]:
sns.lineplot(data=r)